In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from time import sleep
import seaborn as sns
import pandas as pd
import pprint
import numpy as np
from scipy import stats

In [ ]:
data = load_dataset("allenai/WildChat-1M")["train"]["turn"]

In [ ]:
df = data["train"].to_pandas()

In [ ]:
stats.percentileofscore(data, 102)

In [ ]:
pd.Series(data).quantile(.99999)

In [ ]:
len(df[df.turn <103])/len(df)

In [ ]:
df.turn.describe()

In [ ]:
longest = df[df["language"] == "English"].sort_values("turn", ascending=False).head(15)

In [ ]:
longest

In [ ]:
for chat in longest["conversation"].tolist():
    for turn in chat:
        print(f"{turn['role']}: {turn['content']}")
        print()
    print("=======================")
    print()
    sleep(1)

In [ ]:
data = data["train"].remove_columns(['conversation_hash', 'model', 'timestamp', 'turn', 'language', 'openai_moderation', 'detoxify_moderation', 'toxic', 'redacted', 'state', 'country', 'hashed_ip', 'header'])

In [ ]:
model_id = "google/gemma-3-27b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
def encode(examples):
    example =  [[{"content": y["content"], "role": y["role"]}  for y in example] for example in examples["conversation"] ]
    chat = tokenizer.apply_chat_template(example, tokenize=True)
    return {"length": [len(tokens) for tokens in chat]}

In [ ]:
import json

michael = json.load(open("generations/gemma-3-4b-it/alpaca-eval_Michael_Scott_79.json", "r"))

In [ ]:
michael[559].rstrip(" _\n")

In [ ]:
data = data.map(encode, batched=True)

In [ ]:
pd.Series(data["length"]).describe()

In [ ]:
curated = load_dataset("HannahRoseKirk/prism-alignment", 'conversations')

In [ ]:
df = curated["train"].to_pandas()

In [ ]:
df

In [ ]:
df = df.drop_duplicates("opening_prompt")

In [ ]:
df = df[df.conversation_type == "unguided"]

In [ ]:
df = df.drop_duplicates("user_id")

In [ ]:
df = df[~df.opening_prompt.str.contains("hi", case=False)][~df.opening_prompt.str.contains("hey", case=False)][~df.opening_prompt.str.contains("hello", case=False)][~df.opening_prompt.str.contains("helo", case=False)][~df.opening_prompt.str.contains("afternoon", case=False)][~df.opening_prompt.str.contains("trump", case=False)][~df.opening_prompt.str.contains("role", case=False)][~df.opening_prompt.str.contains("kill", case=False)][~df.opening_prompt.str.contains("bmw", case=False)]

In [ ]:
prompts = [x for x in df.sample(200, random_state=42).opening_prompt.tolist()]

In [ ]:
prompts